# **Praca z czasem i strefami czasowymi w Pythonie: `datetime`, `zoneinfo` i `pytz`**

Python oferuje potężne narzędzia do pracy z czasem i datami, w tym moduły `datetime`, `zoneinfo` i opcjonalnie `pytz`. Zrozumienie tych narzędzi jest kluczowe przy zarządzaniu czasem w aplikacjach, szczególnie w kontekście stref czasowych i przeliczeń między różnymi formatami czasu.

---

## Tworzenie obiektów `datetime`

### Tworzenie daty i czasu z wykorzystaniem `ZoneInfo`
Obiekty `datetime` można tworzyć ręcznie, podając rok, miesiąc, dzień oraz opcjonalnie godzinę, minutę, sekundę, mikrosekundę i informację o strefie czasowej (`tzinfo`).


In [1]:
from datetime import datetime
from zoneinfo import ZoneInfo

# Data naive (bez strefy czasowej)
naive_date = datetime(2024, 11, 28, 15, 30)
print("Naive date:", naive_date)

# Data aware (ze strefą czasową)
aware_date = datetime(2024, 11, 28, 15, 30, tzinfo=ZoneInfo("UTC"))
print("Aware date:", aware_date)

# Data aware - Warszawa - przesilenie wiosenne

dt1 = datetime(2024, 3, 31, 2, 30, tzinfo=ZoneInfo("Europe/Warsaw"))
print("Data aware - Warszawa - przesilenie wiosenne:", dt1)
print("tzname:", dt1.tzname())

Naive date: 2024-11-28 15:30:00
Aware date: 2024-11-28 15:30:00+00:00
Data aware - Warszawa - przesilenie wiosenne: 2024-03-31 02:30:00+01:00
tzname: CET


### Tworzenie daty i czasu z wykorzystaniem `pytz`

Po pierwsze trzeba zainstalować `pytz`:

```bash
pip install pytz
```

Daty można tworzyć z wykorzystaniem `pytz` w następujący sposób:

In [2]:
from datetime import datetime
from pytz import timezone

warsaw_tz = timezone('Europe/Warsaw')

# Sposób 1 - bezpośrednie użycie tzinfo (potencjalnie problematyczne)
dt1 = datetime(2024, 3, 31, 2, 30, tzinfo=warsaw_tz)
print("Sposób 1 (nieprawidłowy):", dt1)  # To może nie uwzględnić poprawnie zmiany czasu

# Sposób 2 - użycie localize (prawidłowy sposób)
dt2 = warsaw_tz.localize(datetime(2024, 3, 31, 2, 30))
print("Sposób 2 (prawidłowy):", dt2)

# Sprawdźmy różnicę
print("\nRóżnica w strefach czasowych:")
print("Sposób 1 tzname:", dt1.tzname())
print("Sposób 2 tzname:", dt2.tzname())

# Ta data jest szczególna, bo wypada podczas zmiany czasu z zimowego na letni
# W Polsce 31 marca 2024 o 2:00 następuje zmiana na 3:00


Sposób 1 (nieprawidłowy): 2024-03-31 02:30:00+01:24
Sposób 2 (prawidłowy): 2024-03-31 02:30:00+01:00

Różnica w strefach czasowych:
Sposób 1 tzname: LMT
Sposób 2 tzname: CET


Dokumentacja zaleca użycie `localize` do tworzenia dat aware. Ale kluczowe też jest to, by wewnętrznie pracować z datami w UTC, a localize używać wtedy gdy chcemy wyświetlić datę w jakiejś konkretnej strefie czasowej.

Bazując na przykładzie z [dokumentacji](https://pythonhosted.org/pytz/) `pytz` można zauważyć, że `ZoneInfo` lepiej obsługuj nie tylko zmiany czasu letniego, ale też zmiany historyczne, takie jak zmiana czasu w Warszawie w 1915 roku.


<blockquote>
A special case is where countries change their timezone definitions with no daylight savings time switch. For example, in 1915 Warsaw switched from Warsaw time to Central European time with no daylight savings transition. So at the stroke of midnight on August 5th 1915 the clocks were wound back 24 minutes creating an ambiguous time period that cannot be specified without referring to the timezone abbreviation or the actual UTC offset. In this case midnight happened twice, neither time during a daylight saving time period. pytz handles this transition by treating the ambiguous period before the switch as daylight savings time, and the ambiguous period after as standard time.
</blockquote>


In [18]:
fmt = '%Y-%m-%d %H:%M:%S %Z%z'
warsaw = timezone('Europe/Warsaw')
amb_dt1 = warsaw.localize(datetime(1915, 8, 4, 23, 59, 59), is_dst=True)
print(amb_dt1.strftime(fmt)) # '1915-08-04 23:59:59 WMT+0124'

amb_dt2 = warsaw.localize(datetime(1915, 8, 4, 23, 59, 59), is_dst=False)
print(amb_dt2.strftime(fmt)) # '1915-08-04 23:59:59 CET+0100'

switch_dt = warsaw.localize(datetime(1915, 8, 5, 00, 00, 00), is_dst=False)
switch_dt.strftime(fmt) # '1915-08-05 00:00:00 CET+0100'

print(switch_dt - amb_dt1) # '0:24:01'
print(switch_dt - amb_dt2) #'0:00:01'



1915-08-04 23:59:59 WMT+0124
1915-08-04 23:59:59 CET+0100
0:24:01
0:00:01


In [19]:
from datetime import datetime
from zoneinfo import ZoneInfo

fmt = "%Y-%m-%d %H:%M:%S %Z%z"

# Czas w Warszawie przed zmianą
warsaw = ZoneInfo("Europe/Warsaw")
amb_dt1 = datetime(1915, 8, 4, 23, 59, 59, tzinfo=warsaw)
print(amb_dt1.strftime(fmt))  # '1915-08-04 23:59:59 LMT+0124' (WMT: Warsaw Mean Time)

# Czas po zmianie
switch_dt = datetime(1915, 8, 5, 0, 0, 0, tzinfo=warsaw)
print(switch_dt.strftime(fmt))  # '1915-08-05 00:00:00 CET+0100'

print(switch_dt - amb_dt1) # '0:00:01'
print(switch_dt - amb_dt2) #'0:00:01'

1915-08-04 23:59:59 WMT+0124
1915-08-05 00:00:00 CET+0100
0:00:01
0:00:01


### Podsumowanie róznic między `pytz` i `zoneinfo`


#### **1. Pochodzenie i integracja z Pythonem**
- **`pytz`**: Zewnętrzna biblioteka, która korzysta z bazy danych IANA Time Zone Database. Była standardem przed wprowadzeniem `zoneinfo`.
- **`zoneinfo`**: Wbudowana w Python od wersji 3.9, również korzysta z bazy IANA i jest obecnie rekomendowana dla nowych projektów.

---

#### **2. Obsługa czasu letniego (DST) i niejednoznaczności**
- **`pytz`**: 
  - Wymaga jawnego określenia, czy czas dotyczy DST, za pomocą argumentu `is_dst` w metodzie `localize`. 
  - Daje pełną kontrolę nad obsługą niejednoznacznych dat, np. podczas zmiany czasu letniego.
  - **Przykład:**
    ```python
    from pytz import timezone
    from datetime import datetime

    warsaw = timezone("Europe/Warsaw")
    ambiguous_dt = datetime(2023, 10, 29, 2, 30)  # Niejednoznaczny czas
    dt_dst = warsaw.localize(ambiguous_dt, is_dst=True)  # DST
    dt_standard = warsaw.localize(ambiguous_dt, is_dst=False)  # Czas standardowy
    print(dt_dst)       # 2023-10-29 02:30:00+02:00
    print(dt_standard)  # 2023-10-29 02:30:00+01:00
    ```

- **`zoneinfo`**:
  - Automatycznie obsługuje niejednoznaczności, ale nie daje możliwości ręcznego ustawienia DST.
  - Może zgłaszać wyjątki w przypadku niejednoznacznych dat.
  - **Przykład:**
    ```python
    from zoneinfo import ZoneInfo
    from datetime import datetime

    ambiguous_dt = datetime(2023, 10, 29, 2, 30, tzinfo=ZoneInfo("Europe/Warsaw"))
    print(ambiguous_dt)  # Automatycznie przypisuje offset (np. +01:00 lub +02:00)
    ```

---

#### **3. Wygoda użycia**
- **`pytz`**:
  - Wymaga użycia metody `localize` dla dat bez stref czasowych i `normalize` do konwersji między strefami.
  - Jest bardziej skomplikowane w obsłudze, szczególnie dla początkujących.
  - **Przykład konwersji stref:**
    ```python
    warsaw = timezone("Europe/Warsaw")
    ny = timezone("America/New_York")
    dt = warsaw.localize(datetime(2023, 11, 29, 15, 0))
    converted = dt.astimezone(ny)
    print(converted)  # Czas w strefie Nowego Jorku
    ```

- **`zoneinfo`**:
  - Automatyczne przypisywanie stref czasowych bez dodatkowych metod.
  - Bardziej intuicyjne i prostsze w użyciu.
  - **Przykład konwersji stref:**
    ```python
    dt = datetime(2023, 11, 29, 15, 0, tzinfo=ZoneInfo("Europe/Warsaw"))
    converted = dt.astimezone(ZoneInfo("America/New_York"))
    print(converted)
    ```

---

#### **4. Obsługa historycznych danych**
- Obie biblioteki korzystają z bazy danych IANA, więc mają podobną dokładność w obsłudze historycznych zmian stref czasowych.

---

#### **5. Zgodność z Pythonem**
- **`pytz`**: Wymaga instalacji zewnętrznej (`pip install pytz`).
- **`zoneinfo`**: Wbudowana w Python od wersji 3.9, nie wymaga instalacji.

---

#### **6. Rekomendacja**
- **`pytz`**: Dobra dla starszych projektów, które już ją wykorzystują, lub w przypadkach, gdy potrzebna jest pełna kontrola nad DST.
- **`zoneinfo`**: Rekomendowana dla nowych projektów dzięki prostocie, intuicyjności i integracji z Pythonem.

---

#### **Podsumowanie różnic w tabeli**

| **Cecha**                | **pytz**                                     | **zoneinfo**                                |
|---------------------------|----------------------------------------------|--------------------------------------------|
| **Instalacja**            | Zewnętrzna biblioteka (`pip install pytz`)   | Wbudowana w Python 3.9+                    |
| **Obsługa DST**           | Ręczna kontrola za pomocą `is_dst`          | Automatyczna                                |
| **Przypisywanie stref**   | `localize` i `normalize` wymagane            | Wystarczy `ZoneInfo`                       |
| **Obsługa niejednoznaczności** | Pełna kontrola                          | Automatyczne przypisanie (bez `is_dst`)     |
| **Wygoda użycia**         | Bardziej skomplikowane                      | Prostsze i bardziej intuicyjne             |
| **Obsługa historyczna**   | Pełna                                       | Pełna                                       |

Jeśli zaczynasz nowy projekt, wybierz **`zoneinfo`**. Jeśli pracujesz nad starszym kodem lub potrzebujesz kontroli nad DST, pozostanie przy **`pytz`** może być uzasadnione.

## Offset

Offset czasu to różnica w czasie między daną strefą czasową a czasem uniwersalnym (UTC). Offset jest podawany w formacie `±HH:MM`, gdzie:

- **`+01:00`**: Lokalny czas jest godzinę do przodu względem UTC.
- **`-05:00`**: Lokalny czas jest 5 godzin za UTC.

Offset jest ważny, gdy pracujemy z datami i godzinami uwzględniającymi strefy czasowe, np. `2024-11-29 15:00:00+01:00`.



---

### **Tworzenie obiektów `datetime` z offsetem**

W Pythonie można ustawić offset dla obiektu `datetime` przy użyciu klasy `timezone` z modułu `datetime`.




In [30]:

from datetime import datetime, timezone, timedelta

# Tworzenie offsetu
offset = timezone(timedelta(hours=1))  # +01:00

# Tworzenie obiektu datetime z offsetem
dt = datetime(2024, 11, 29, 15, 0, 0, tzinfo=offset)
print(dt)  # 2024-11-29 15:00:00+01:00


2024-11-29 15:00:00+01:00


Offsety mogą być dodatnie lub ujemne.

In [31]:

# Offset dodatni (+02:30)
offset_positive = timezone(timedelta(hours=2, minutes=30))
dt_positive = datetime(2024, 11, 29, 12, 0, 0, tzinfo=offset_positive)
print(dt_positive)  # 2024-11-29 12:00:00+02:30

# Offset ujemny (-08:00)
offset_negative = timezone(timedelta(hours=-8))
dt_negative = datetime(2024, 11, 29, 4, 0, 0, tzinfo=offset_negative)
print(dt_negative)  # 2024-11-29 04:00:00-08:00



2024-11-29 12:00:00+02:30
2024-11-29 04:00:00-08:00


Jeśli mamy obiekt `datetime` w jednej strefie czasowej i chcemy go przekonwertować na inną, możemy użyć metody `.astimezone()`.

In [32]:
# Tworzenie daty w strefie +01:00
dt = datetime(2024, 11, 29, 15, 0, 0, tzinfo=timezone(timedelta(hours=1)))

# Konwersja na UTC
utc = timezone.utc
dt_utc = dt.astimezone(utc)
print(dt_utc)  # 2024-11-29 14:00:00+00:00

# Konwersja na inną strefę (np. -05:00)
offset_minus_5 = timezone(timedelta(hours=-5))
dt_minus_5 = dt.astimezone(offset_minus_5)
print(dt_minus_5)  # 2024-11-29 09:00:00-05:00


2024-11-29 14:00:00+00:00
2024-11-29 09:00:00-05:00


`zoneinfo` pozwala na automatyczne przypisywanie offsetu w zależności od strefy czasowej i daty.

In [34]:

from datetime import datetime
from zoneinfo import ZoneInfo

# Tworzenie daty w strefie Europe/Warsaw
dt_warsaw = datetime(2024, 11, 29, 15, 0, 0, tzinfo=ZoneInfo("Europe/Warsaw"))
print(dt_warsaw)  # 2024-11-29 15:00:00+01:00

# Tworzenie daty w strefie America/New_York
dt_new_york = datetime(2024, 11, 29, 9, 0, 0, tzinfo=ZoneInfo("America/New_York"))
print(dt_new_york)  # 2024-11-29 09:00:00-05:00


2024-11-29 15:00:00+01:00
2024-11-29 09:00:00-05:00


## **Przedstawianie `datetime` w postaci napisu w Pythonie**

Obsługa dat i czasu to jeden z kluczowych aspektów w programowaniu, a Python oferuje wiele narzędzi do pracy z obiektami daty i czasu. Jednym z najczęstszych zadań jest konwersja obiektu `datetime` na ciąg znaków (`string`). Proces ten umożliwia łatwe wyświetlanie lub zapis daty i czasu w czytelnym formacie.

---

### **Metoda `strftime`**

Funkcja `strftime` (ang. *string format time*) z modułu `datetime` umożliwia sformatowanie daty i czasu na różne sposoby. Używa się jej z określonymi symbolami formatu, które definiują wygląd wyniku.

#### **Podstawowe symbole formatowania:**
- `%Y` – Rok (pełny, np. 2024).
- `%m` – Miesiąc (01–12).
- `%d` – Dzień miesiąca (01–31).
- `%H` – Godzina (00–23).
- `%M` – Minuty (00–59).
- `%S` – Sekundy (00–59).
- `%z` – Offset UTC, np. `+0100`.
- `%Z` – Nazwa strefy czasowej, np. `CET`.

#### **Przykład użycia:**

```python
from datetime import datetime

now = datetime.now()  # Aktualna data i czas
formatted = now.strftime("%Y-%m-%d %H:%M:%S")
print(formatted)  # Przykład: 2024-11-29 15:45:30
```

---

### **Obsługa stref czasowych**

Jeśli obiekt `datetime` zawiera informacje o strefie czasowej, można uwzględnić je w wyniku:

```python
from datetime import datetime, timezone, timedelta

# Tworzenie daty z offsetem UTC
dt = datetime(2024, 11, 29, 15, 0, tzinfo=timezone(timedelta(hours=1)))

# Formatowanie z offsetem
formatted = dt.strftime("%Y-%m-%d %H:%M:%S %z")
print(formatted)  # 2024-11-29 15:00:00 +0100
```

---

### **Rzutowanie `datetime` na napis**

Python pozwala na szybkie rzutowanie obiektu `datetime` na ciąg znaków przy użyciu funkcji `str()`:

```python
from datetime import datetime

now = datetime.now()
print(str(now))  # Przykład: 2024-11-29 15:45:30.123456
```

Domyślny format zawiera sekundy z mikrosekundami, ale nie uwzględnia strefy czasowej, jeśli obiekt jej nie posiada.

---

### **Konwersja do ISO 8601**

Standard **ISO 8601** to popularny format do przechowywania i wymiany dat w systemach informatycznych. W Pythonie można łatwo uzyskać taki format za pomocą metody `.isoformat()`:

```python
from datetime import datetime, timezone

now = datetime.now(timezone.utc)
print(now.isoformat())  # Przykład: 2024-11-29T15:45:30+00:00
```

---

### **Podsumowanie**

Przedstawianie obiektów `datetime` jako napisu w Pythonie jest elastyczne i dostosowane do różnych potrzeb. Najczęściej wykorzystywane metody to:

1. **`strftime`** – pełna kontrola nad formatowaniem.
2. **`str()`** – szybkie rzutowanie na czytelny ciąg znaków.
3. **`.isoformat()`** – formatowanie zgodne z ISO 8601.

Wybór odpowiedniej metody zależy od wymagań projektu – czy potrzebujemy standardowego formatu, czy niestandardowego układu dla użytkownika. Dzięki wbudowanym narzędziom Python sprawia, że proces ten jest intuicyjny i wszechstronny.



---

## **1.2. Tworzenie z napisów**
Daty można tworzyć z tekstu za pomocą `strptime`.


In [29]:

# Tworzenie z napisu - data bez strefy czasowej
date_from_string = datetime.strptime("2024-11-28 15:30:00", "%Y-%m-%d %H:%M:%S")
print("Z napisu:", date_from_string)

# tworzenie z napisu - data z strefą czasową jako offset
utc_date = datetime.strptime("2024-11-28 15:30:00 +0000", "%Y-%m-%d %H:%M:%S %z")
print("Aware date (UTC):", utc_date)
print(utc_date.tzname(), utc_date.tzinfo)

# tworzenie z napisu - data z strefą czasową jako offset
utc_plus_1_date = datetime.strptime("2024-11-28 15:30:00 +0100", "%Y-%m-%d %H:%M:%S %z")
print("Aware date (UTC+1):", utc_plus_1_date)
print(utc_plus_1_date.tzname(), utc_plus_1_date.tzinfo)

date_from_string_aware = datetime.strptime("2024-11-28 15:30:00", "%Y-%m-%d %H:%M:%S").astimezone(ZoneInfo("Europe/Warsaw"))
print("Z napisu - aware:", date_from_string_aware)
print(date_from_string_aware.tzname(), date_from_string_aware.tzinfo)


Z napisu: 2024-11-28 15:30:00
Aware date (UTC): 2024-11-28 15:30:00+00:00
UTC UTC
Aware date (UTC+1): 2024-11-28 15:30:00+01:00
UTC+01:00 UTC+01:00
Z napisu - aware: 2024-11-28 15:30:00+01:00
CET Europe/Warsaw





### **1.3. Tworzenie z timestamp**
**Timestamp** to liczba sekund od **epoch** (1 stycznia 1970, 00:00:00 UTC w systemach Unix).

```python
# Z timestamp
timestamp = 1701171000  # Przykład timestamp
date_from_timestamp = datetime.fromtimestamp(timestamp, tz=ZoneInfo("UTC"))
print("Z timestamp:", date_from_timestamp)
```

---

## **2. Obsługa stref czasowych**

Python od wersji 3.9 oferuje moduł `zoneinfo`, który automatycznie obsługuje strefy czasowe i zmiany czasu letniego (DST). Można również korzystać z `pytz` w starszych wersjach Pythona.

### **2.1. Tworzenie obiektów aware**
Obiekty `aware` zawierają informację o strefie czasowej.

```python
from zoneinfo import ZoneInfo

utc_date = datetime(2024, 11, 28, 15, 30, tzinfo=ZoneInfo("UTC"))
warsaw_date = utc_date.astimezone(ZoneInfo("Europe/Warsaw"))
print("Data UTC:", utc_date)
print("Data Warsaw:", warsaw_date)
```

### **2.2. Przeliczanie między strefami**
Metoda `astimezone` pozwala na konwersję czasu między strefami czasowymi.

```python
# Konwersja do innej strefy czasowej
new_york_date = warsaw_date.astimezone(ZoneInfo("America/New_York"))
print("Data New York:", new_york_date)
```

---

## **3. Czas letni (DST)**

Zmiany czasu letniego są automatycznie obsługiwane przez `zoneinfo`.

```python
from datetime import timedelta

# Data podczas czasu letniego
summer_date = datetime(2024, 7, 1, 12, tzinfo=ZoneInfo("Europe/Warsaw"))
print("Summer date:", summer_date)

# Data po przejściu na czas zimowy
winter_date = summer_date + timedelta(days=150)
print("Winter date:", winter_date)
```

---

## **4. Timestamp i różne epoki**

### **4.1. Różne epoch**
Różne systemy mogą używać różnych punktów odniesienia (epoch):
- **Unix epoch**: 1 stycznia 1970, 00:00:00 UTC.
- **Windows epoch**: 1 stycznia 1601, 00:00:00 UTC.

### **4.2. Operacje z timestamp**
```python
# Tworzenie timestamp
current_timestamp = datetime.now(tz=ZoneInfo("UTC")).timestamp()
print("Aktualny timestamp:", current_timestamp)

# Tworzenie daty z Unix epoch
date_from_unix_epoch = datetime.fromtimestamp(0, tz=ZoneInfo("UTC"))
print("Data z Unix epoch:", date_from_unix_epoch)
```

---

## **5. `zoneinfo` vs. `pytz`**

| **Cecha**                  | **`zoneinfo`**            | **`pytz`**             |
|----------------------------|--------------------------|------------------------|
| **Wbudowana w bibliotekę** | Tak (od Python 3.9)      | Nie, wymaga instalacji |
| **Obsługa DST**            | Automatyczna            | Wymaga jawnych konwersji |
| **Intuicyjność**           | Wyższa                  | Niższa                 |

Przykład z `pytz`:

```python
import pytz

# Ustawianie strefy czasowej
warsaw_tz = pytz.timezone("Europe/Warsaw")
date_pytz = warsaw_tz.localize(datetime(2024, 11, 28, 15, 30))
print("pytz date:", date_pytz)
```

---

## **6. Ćwiczenia**

1. **Z timestamp do stref czasowych**  
   Napisz funkcję, która przyjmuje timestamp i listę stref czasowych, a następnie zwraca czas w każdej z tych stref.

2. **Rozpoznanie czasu letniego**  
   Napisz funkcję, która sprawdza, czy podana data w określonej strefie czasowej przypada na czas letni.

---

## **Podsumowanie**

Moduł `datetime` w Pythonie, wraz z `zoneinfo`, pozwala na efektywne zarządzanie czasem w aplikacjach globalnych. Obsługa timestamp, konwersje między strefami czasowymi oraz automatyczna obsługa czasu letniego czynią go idealnym narzędziem do pracy z datami. `zoneinfo` jest intuicyjny i nowoczesny, ale dla starszych projektów można używać `pytz`. Wybór zależy od wersji Pythona i wymagań projektu.